<a href="https://colab.research.google.com/github/Lovette-Duke/web_scraping_projects/blob/main/scrape_sg_job_links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
!apt-get update
!apt install chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,582 kB]
Get:14 http://security.ubuntu.com/ubuntu j

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import time

from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
job_links = pd.read_csv('sg_jobs_links.csv')
# sg_jobs = pd.read_csv('sg_jobs.csv')
job_links = job_links['Links'].tolist()
len(job_links)

7394

In [ ]:
del job_titles[1200:]
del company_names[1200:]
del posted_dates[1200:]

In [ ]:
driver = web_driver()
for link in job_links[6800:7000]:
  driver.get(link)
  time.sleep(5)

  # Get Job Tittle
  try:
    job_title = driver.find_element(By.CSS_SELECTOR, 'h1.job-title').text
    job_titles.append(job_title)
  except:
    job_titles.append(None)


  # Get Company Name
  try:
    company_name = driver.find_element(By.CSS_SELECTOR, 'span.company').text
    company_names.append(company_name)
  except:
    company_names.append(None)

  # Get Posted Date
  try:
    span_tag = driver.find_element(By.CSS_SELECTOR, 'span.listed-date')
    # extract the time text from the span tag
    posted_text = span_tag.text.strip()  # e.g., "Posted 3h ago"
    posted_dates.append(posted_text)
    # match = re.search(r'Posted (\d+)([hd]) ago', posted_text)

    # if match:
    #   number = int(match.group(1))
    #   unit = match.group(2)
    #   # get the current datetime
    #   now = datetime.now()
    #   # calculate the posted date based on the unit
    #   if unit == 'h':  # hours
    #       posted_date = now - timedelta(hours=number)
    #   elif unit == 'd':  # days
    #       posted_date = now - timedelta(days=number)
    #   else:
    #       posted_date = now  # default to now if unit not recognized
    #   # format the date as desired
    #   formatted_date = posted_date.strftime('%Y-%m-%d')
    #   posted_dates.append(formatted_date)
    # else:
    #   posted_dates.append(None)
  except:
    posted_dates.append(None)

driver.quit()

In [ ]:
len(posted_dates)

2000

In [ ]:
posted_dates[1990:]

['9 days ago',
 '9 days ago',
 '13 days ago',
 '13 days ago',
 '12 days ago',
 '12 days ago',
 '14 days ago',
 '14 days ago',
 '4 months ago',
 '4 months ago']

In [ ]:
df = pd.DataFrame({
    "Company Name": company_names,
    "Job Title": job_titles,
    "Posted Date": posted_dates
})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company Name  1070 non-null   object
 1   Job Title     2000 non-null   object
 2   Posted Date   2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [ ]:
sg_df = pd.read_csv('sg_jobs.csv')
sg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5394 entries, 0 to 5393
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company Name  3036 non-null   object
 1   Job Title     5394 non-null   object
 2   Posted Date   5394 non-null   object
dtypes: object(3)
memory usage: 126.5+ KB


In [ ]:
df_concat = pd.concat([sg_df, df], ignore_index=True)
df_concat.reset_index(drop=True, inplace=True)
df_concat.to_csv('sg_jobs.csv', index=False)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7394 entries, 0 to 7393
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Company Name  4106 non-null   object
 1   Job Title     7394 non-null   object
 2   Posted Date   7394 non-null   object
dtypes: object(3)
memory usage: 173.4+ KB


In [ ]:
if "month" in date:
